# An Explain Report

This is a POC for an explain report: given a number at time T1 and time T2, introspect the clocks involved to produce a simple linear breakdown
of what changed on various timelines to account for the change in the number between T1 and T2.

The report is an ordered sequence of clock changes: the sum of changes should equal to the total change. The order of application is important: change the
ordering, and the resultant values may change. It's not a Jacobian-type report with T1 deltas and no ordering.

The real reason for this workbook is to start working on the node class optimizations and metadata management.

## Footnotes

Footnotes are one of the simplest types of metadata:
* Any computation (even a simple read of a constant value) may declare it has footnotes
* Every computation gets the footnote of all its inputs

Footnoting is based on a number of observations about large software systems (say, 1M LOC or more):
* Any complex report (i.e. computation output) will generally be somewhat wrong, misleading, or out of date.
* If you want 100% correctness and truth, your report will raise an exception every time you run it.
* If you just want it to run, you'll get a result, but some log file on some machine in some compute farm will have a message explaining why your result is wrong.
* And you won't see that message.
* And that message may have leaked information that bad actors can read.
* And you won't trust this system, so your own developers will copy the underlying data and write their own version of the report with all the same issues.
* Now you don't have a problem, but your company now has two problems.

Footnotes summary:
* Are the documentation of problems from the producer's point of view
* Make no claim about usability of results, etc, for a specific consumer
* May be programmatically removed/condensed/replaced at controlled code points


In [1]:
from mand.core import Entity, node, Context
from mand.core import ObjectDb, Timestamp, getNode, _tr
from mand.core import ProfileMonitor
from mand.core import displayDict, displayMarkdown, displayListOfDicts, displayHeader
from mand.core import find
from mand.demos.trading import makeWorld, bookSomeTrades

db = ObjectDb()

from mand.lib.dbsetup import setUpDb
setUpDb(db)
db.describe()

<mand.db.ObjectDb object at 0x108203d50>: 289, mem=True, ro=False: entities=9, map=2


In [2]:
with db:
    pWorld = makeWorld()
    
pAll, bExt, bExt2 = pWorld.children()
p1 = pAll.children()[0]
p2 = pAll.children()[1]
p4 = pAll.children()[3]

b2 = p2.children()[0]
b4 = p4.children()[0]

makeWorld, TopOfTheHouse is: <Entity:/Global/TradingPortfolio/TopOfTheHouse>
    # books: 100
    # children: 10


# Book some trades

This time, we throw a few thousand in...

In [3]:
ts0, ts1, ts2, ts3, ts4, ts5, eod, ts6 = bookSomeTrades(pWorld)

# An Explain Report

This is a mix of abstractions at the Core, DBA, and BA, and User levels. But, it does the job for now...

In [4]:
class Report(Entity):
    @node(stored=True)
    def valuable(self):
        return None
    
    @node(stored=True)
    def ts1(self):
        return None
    
    @node(stored=True)
    def ts2(self):
        return None
    
    @node
    def data(self):
        valuable = self.valuable()
        ts1 = self.ts1()
        ts2 = self.ts2()
        clock = valuable.getObj(_tr.RootClock, 'Main')
    
        def clocks(ts):
            def fn(node):
                obj = node.key[0]
                m = node.key[1].split(':')[-1]
                if isinstance(obj, _tr.Clock) and m == 'cutoffs':
                    return True
            with Context({clock.cutoffs: ts}, 'Clocks'):
                nodes = find(valuable.NPV, fn)
                return dict( [ (node.tweakPoint, node) for node in nodes ] )
    
        allNodes = clocks(ts1)
    
        allNodes.update(clocks(ts2))
        nodes = allNodes.values() 
    
        # IRL, we'd sort these according to some business req...
        # And our clocks might be arranged in an N-level tree...
        nodes = sorted(nodes, key = lambda node: node.key[0].meta.name())
    
        data = []
        curr = [0]
        def add(title, npv):
            pnl = npv - curr[0]
            curr[0] = npv
            data.append( {'Activity': title, 'PnL': pnl } )

        with Context({clock.cutoffs: ts1}, 'Start'):
            curr = [ valuable.NPV() ] # Starting balance
    
        tweaks = {}
        for n in nodes:
            tweaks[n.tweakPoint] = ts1
        with Context(tweaks, name='Start breaks'):
            start = valuable.NPV()
            add('Starting balance breaks', start)

        tsAmend = Timestamp(t=ts2.transactionTime, v=ts1.validTime)
        # XXX - modifying tweaks in place is a bit evil
        # This is only safe because I know Context() effectively copies, so this works
        # for now.
        for n in nodes:
            tweaks[n.tweakPoint] = tsAmend
            name = n.key[0].meta.name()
            with Context(tweaks, name='Amend %s' % name):
                add('prior day amends: %s' % name, valuable.NPV())
        for n in nodes:
            tweaks[n.tweakPoint] = ts2
            name = n.key[0].meta.name()
            with Context(tweaks, name='Activity %s' % name):
                add('activity: %s' % name, valuable.NPV())
    
        with Context({clock.cutoffs: ts2}, name='End'):
            end = valuable.NPV()
            add('Ending balance breaks', end)

        title = 'PnL explain for %s: %s' % (valuable.meta.name(), end-start)
        return data, title

    def run(self):
        data, title = self.data()
        node = getNode(self.data)
        footnotes = node.footnotes.values()
        displayHeader('%s' % title)
        if footnotes:
            displayMarkdown('**Caveat: this report encountered problems. See footnotes at bottom.**')
        displayListOfDicts(data, names=['Activity', 'PnL'] )
        if footnotes:
            displayMarkdown('## Footnotes:')
            txt = '\n'.join( [ '1. %s' % f for f in footnotes])
            displayMarkdown(txt)
    
r = Report(valuable=pAll, ts1=eod, ts2=ts6)
r.run()

# PnL explain for TopOfTheHouse: 5236.00

**Caveat: this report encountered problems. See footnotes at bottom.**

|Activity|PnL|
|-|-|
|Starting balance breaks|0.00
|prior day amends: MarketData|0.00
|prior day amends: Portfolio|0.00
|prior day amends: Trading|-4.00
|activity: MarketData|5240.00
|activity: Portfolio|0.00
|activity: Trading|0.00
|Ending balance breaks|0.00

## Footnotes:

1. Inadequate cash discounting model used

## Add some inconsistent data [Test]

Book b2 should appear multiple times in some portfolio trees and be flagged accordingly...

In [5]:
with db:
    p1.setChildren(p1.children() + [b2])

ts7 = Timestamp()

# Footnotes

Note the report calculation has run, but attached appropriate caveats to the output:

In [6]:
db3 = db.copy()
p = db3._get(pAll.meta.path())

with ProfileMonitor(mode='sum'): 
    r = Report(valuable=p, ts1=eod, ts2=ts7)
    r.run()

# PnL explain for TopOfTheHouse: 6256.00

**Caveat: this report encountered problems. See footnotes at bottom.**

|Activity|PnL|
|-|-|
|Starting balance breaks|0.00
|prior day amends: MarketData|0.00
|prior day amends: Portfolio|0.00
|prior day amends: Trading|-4.00
|activity: MarketData|5240.00
|activity: Portfolio|1020.00
|activity: Trading|0.00
|Ending balance breaks|0.00

## Footnotes:

1. Inadequate cash discounting model used
1. Book appears multiple times: /Global/TradingBook/Eq-Inst0


### Profile by nodes.
* times are in microseconds
* cumT is total time spent in funtion
* calcT is time spent in function, but not in a child node

|fn|n|cumT|calcT|cumT/call|sys|
|-|-|-|-|-|-|
|Report:data|1|258,823,125|62|258,823,125|GetValue
|Report:data|1|258,823,062|2,557|258,823,062|GetValue/Calc
|TradingContainer:NPV|11|258,248,354|575|23,477,123|GetValue
|TradingContainer:NPV|11|258,247,779|26,570|23,477,070|GetValue/Calc
|Portfolio:items|121|256,438,134|25,923|2,119,323|GetValue
|Portfolio:items|121|256,426,929|346,052|2,119,230|GetValue/Calc
|Workbook:items|1,104|242,784,600|136,756|219,913|GetValue
|Workbook:items|1,100|242,647,844|95,756,482|220,588|GetValue/Calc
|Root:Clocks|2|175,330,229|520,442|87,665,114|Context
|TradeOpenEvent|1,014|72,290,138|72,290,138|71,292|Db.Get
|TradeOpenEvent:ticket|22,286|65,732,509|1,492,849|2,949|GetValue
|TradingTicket|1,014|64,239,660|64,239,660|63,352|Db.Get
|Root:Start breaks|1|14,187,670|320|14,187,670|Context
|Root:Start|1|13,826,335|144|13,826,335|Context
|Portfolio:children|242|13,216,193|27,865|54,612|GetValue
|Portfolio:children|121|13,188,327|4,904,972|108,994|GetValue/Calc
|Root:Amend MarketData|1|9,506,510|168|9,506,510|Context
|Root:Activity Portfolio|1|8,971,668|184|8,971,668|Context
|Root:Amend Portfolio|1|7,581,150|157|7,581,150|Context
|Root:Activity Trading|1|7,430,976|174|7,430,976|Context
|Root:Amend Trading|1|7,326,753|160|7,326,753|Context
|Root:End|1|7,317,962|175|7,317,962|Context
|Root:Activity MarketData|1|7,291,185|160|7,291,185|Context
|PortfolioUpdateEvent:children|121|7,098,628|26,449|58,666|GetValue
|TradingBook|102|6,623,943|6,623,943|64,940|Db.Get
|Equity:NPV|15|1,771,323|745|118,088|GetValue
|Equity:NPV|15|1,770,578|4,334|118,038|GetValue/Calc
|MarketInterface:spot|15|1,670,260|531|111,350|GetValue
|MarketInterface:spot|15|1,669,728|16,104|111,315|GetValue/Calc
|ExternalRefData:state|15|1,574,965|589|104,997|GetValue
|ExternalRefData:state|15|1,574,376|3,693|104,958|GetValue/Calc
|RefData:state|15|1,570,682|10,930|104,712|GetValue
|RefData:state|15|1,559,752|601,718|103,983|GetValue/Calc
|_WorkItemEvent:book2|22,286|1,333,023|1,231,566|59|GetValue
|_WorkItemEvent:book1|22,286|1,215,919|1,215,919|54|GetValue
|Clock:cutoffs|2,480|1,184,062|135,618|477|GetValue
|TradeOpenEvent:quantity|22,286|1,166,521|1,166,521|52|GetValue
|Clock:cutoffs|20|1,048,849|12,542|52,442|GetValue/Calc
|Clock:parent|20|1,035,760|766|51,788|GetValue
|Clock:parent|20|1,034,994|800,178|51,749|GetValue/Calc
|_WorkItemEvent:item|11,143|1,001,304|904,778|89|GetValue
|PortfolioUpdateEvent|12|926,732|926,732|77,227|Db.Get
|TradeOpenEvent:action|11,143|923,117|923,117|82|GetValue
|TradeOpenEvent:premium|11,143|784,259|734,864|70|GetValue
|TradeOpenEvent:unitPrice|11,143|748,270|748,270|67|GetValue
|Event:amends|11,340|615,581|615,581|54|GetValue
|TradingPortfolio|10|549,691|549,691|54,969|Db.Get
|TradingBook:clock|2,200|508,054|222,547|230|GetValue
|RefDataUpdateEvent|9|469,370|469,370|52,152|Db.Get
|TradingBook:clock|1,100|285,507|228,758|259|GetValue/Calc
|Clock|5|265,661|265,661|53,132|Db.Get
|ClockEvent|2|117,113|117,113|58,556|Db.Get
|ClockEvent:parent|8|112,828|7,268|14,103|GetValue
|Equity|2|96,526|96,526|48,263|Db.Get
|Equity:refdata|15|95,983|645|6,398|GetValue
|Equity:refdata|15|95,337|12,143|6,355|GetValue/Calc
|Portfolio:clock|242|85,493|10,966|353|GetValue
|MarketInterface|2|82,792|82,792|41,396|Db.Get
|MarketInterface:source|15|78,659|540|5,243|GetValue
|MarketInterface:source|15|78,119|8,013|5,207|GetValue/Calc
|Portfolio:clock|121|74,526|7,675|615|GetValue/Calc
|MarketDataSource|2|69,344|69,344|34,672|Db.Get
|Portfolio:books|231|67,068|6,811|290|GetValue
|Portfolio:books|121|61,597|58,552|509|GetValue/Calc
|RootClock|1|49,996|49,996|49,996|Db.Get
|ForwardCashflow|1|49,395|49,395|49,395|Db.Get
|MarketDataSource:clock|30|48,918|11,511|1,630|GetValue
|MarketDataSource:clock|15|37,407|905|2,493|GetValue/Calc
|ForwardCashflow:NPV|11|11,750|719|1,068|GetValue
|ForwardCashflow:NPV|11|11,031|11,031|1,002|GetValue/Calc
|Entity:clock|40|4,124|1,625|103|GetValue
|Entity:clock|20|2,499|2,499|124|GetValue/Calc
|RefDataUpdateEvent:data|58|1,062|1,062|18|GetValue
|MarketInterface:sourceName|15|761|532|50|GetValue
|RootClock:cutoffs|52|669|669|12|GetValue
|Equity:assetName|15|401|401|26|GetValue
|MarketInterface:sourceName|15|228|228|15|GetValue/Calc
|Report:valuable|1|32|32|32|GetValue
|Report:ts1|1|16|16|16|GetValue
|Report:ts2|1|15|15|15|GetValue